In [1]:
# Image to Depth Map
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
from PIL import Image
import torch
from transformers import GLPNFeatureExtractor, GLPNForDepthEstimation
import numpy as np
import open3d as o3d


image_name = "bathroom.jpg"
mesh_name = f"{image_name.rsplit('.', 1 )[ 0 ]}.obj"

# load and resize the input image
image = Image.open(image_name)
new_height = 480 if image.height > 480 else image.height
new_height -= (new_height % 32)
new_width = int(new_height * image.width / image.height)
diff = new_width % 32
new_width = new_width - diff if diff < 16 else new_width + 32 - diff
new_size = (new_width, new_height)
image = image.resize(new_size)

# load model
feature_extractor = GLPNFeatureExtractor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")

# prepare image for the model
inputs = feature_extractor(images=image, return_tensors="pt")

# get the prediction from the model
with torch.no_grad():
    outputs = model(**inputs)
    predicted_depth = outputs.predicted_depth

# remove borders
pad = 16
output = predicted_depth.squeeze().cpu().numpy() * 1000.0
output = output[pad:-pad, pad:-pad]
image = image.crop((pad, pad, image.width - pad, image.height - pad))

# visualize depth map
# fig, ax = plt.subplots(1, 2)
# ax[0].imshow(image)
# ax[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
# ax[1].imshow(output, cmap='plasma')
# ax[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
# plt.tight_layout()
# plt.pause(5)

In [2]:
# Depth Map to Point Cloud
width, height = image.size

depth_image = (output * 255 / np.max(output)).astype('uint8')
image = np.array(image)

# create rgbd
depth_o3d = o3d.geometry.Image(depth_image)
image_o3d = o3d.geometry.Image(image)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(image_o3d, depth_o3d, convert_rgb_to_intensity=False)

# camera
camera = o3d.camera.PinholeCameraIntrinsic()
camera.set_intrinsics(width, height, 500, 500, width/2, height/2)

# create point cloud
pc = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera)
# o3d.visualization.draw_geometries([pc])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Generate Mesh from Point Cloud using Poisson surface reconstruction

# outliers removal
cl, ind = pc.remove_statistical_outlier(nb_neighbors=20, std_ratio=20.0)
pc = pc.select_by_index(ind)

# estimate normals
pc.estimate_normals()
pc.orient_normals_to_align_with_direction()

# surface reconstruction
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pc, depth=10, n_threads=1)[0]

# rotate the mesh
rotation = mesh.get_rotation_matrix_from_xyz((np.pi, 0, 0))
mesh.rotate(rotation, center=(0, 0, 0))

# save the mesh
o3d.io.write_triangle_mesh(mesh_name, mesh)

# visualize the mesh
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)